In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## scale data method--我们在论文中提出的按照七种不同缩放方法和四个不同处理方向的预处理策略

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# #######################
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
)


def ScaleData(train_x, scaling_method, dimension, random_seed):
    if scaling_method == "none":
        return train_x
    if np.isinf(train_x).any():
        print("Train or test set contains infinity.")
        exit(-1)
    scaling_dict = {
        "minmax": MinMaxScaler(),
        "maxabs": MaxAbsScaler(),
        "standard": StandardScaler(),
        "robust": RobustScaler(),
        "quantile": QuantileTransformer(random_state=random_seed),
        "powert": PowerTransformer(),
        "normalize": Normalizer(),
    }
    if scaling_method not in scaling_dict.keys():
        print(f"Scaling method {scaling_method} not found.")
        exit(-1)
    if dimension not in ["timesteps", "channels", "all", "both"]:
        print(f"Dimension {dimension} not found.")
        exit(-1)

    dim1 = -1
    dim2 = 1
    if scaling_method == "normalize":
        dim1 = 1
        dim2 = -1
    out_train_x = np.zeros_like(train_x, dtype=np.float64)

    train_shape = train_x.shape
    if dimension == "all":
        out_train_x = (
            scaling_dict[scaling_method]
            .fit_transform(train_x.reshape((dim1, dim2)))
            .reshape(train_shape)
        )
    else:
        if dimension == "channels":
            train_channel_shape = train_x[:, 0, :].shape
            for i in range(train_x.shape[1]):
                out_train_x[:, i, :] = (
                    scaling_dict[scaling_method]
                    .fit_transform(train_x[:, i, :].reshape((dim1, dim2)))
                    .reshape(train_channel_shape)
                )

        elif dimension == "timesteps":
            train_timest_shape = train_x[:, :, 0].shape

            for i in range(train_x.shape[2]):
                out_train_x[:, :, i] = (
                    scaling_dict[scaling_method]
                    .fit_transform(train_x[:, :, i].reshape((dim1, dim2)))
                    .reshape(train_timest_shape)
                )

        elif dimension == "both":
            train_both_shape = train_x[:, 0, 0].shape
            for i in range(train_x.shape[1]):
                for j in range(train_x.shape[2]):
                    out_train_x[:, i, j] = (
                        scaling_dict[scaling_method]
                        .fit_transform(train_x[:, i, j].reshape((dim1, dim2)))
                        .reshape(train_both_shape)
                    )

        else:
            print(f"Dimension {dimension} not found.")
            exit(-1)
    return out_train_x

##  Import library

In [ ]:
import time
import warnings

import keras.backend as K
import keras.layers as layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.callbacks import LearningRateScheduler
from keras.layers import LSTM, BatchNormalization, Bidirectional, Dense, Dropout
from keras.layers.core import Activation, Dense, Dropout, Flatten
from keras.models import Sequential
from numpy import newaxis


##周期性学习率衰减，调试用
def scheduler(epoch):
    # 每隔5个epoch，学习率减小为原来的1/10-lstm
    # if epoch % 5 == 0 and epoch != 0:
    if epoch % 10 == 0 and epoch != 0:
        # lr = K.get_value(model.optimizer.lr*0.001)#LSTM
        lr = K.get_value(model.optimizer.lr * 10)  # 一维卷积
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)


reduce_lr = LearningRateScheduler(scheduler)

## LSTM

In [ ]:
def built_model_1(Num_Class=None):
    model = Sequential()  # layers [128,64,32,16,4]
    model.add(LSTM(input_shape=(None, 90), units=200, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(input_shape=(None, 200), units=128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(input_shape=(None, 128), units=64, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=Num_Class))
    # model.add(Activation("linear"))
    model.add(Activation("softmax"))
    start = time.time()
    model.compile(
        optimizer="rmsprop",  # 加速神经网络
        loss="categorical_crossentropy",  # 损失函数
        metrics=["accuracy"],
    )
    return model

## FCN 一维卷积

In [ ]:
def built_model_2(Num_Class=None):
    model = Sequential()
    model.add(layers.Convolution1D(128, 3, strides=1))
    model.add(layers.LayerNormalization())
    model.add(Activation("relu"))
    model.add(layers.Convolution1D(256, 3, strides=1))
    model.add(layers.LayerNormalization())
    model.add(Activation("relu"))
    model.add(layers.Convolution1D(128, 3, strides=1))
    model.add(layers.LayerNormalization())
    model.add(Activation("relu"))
    model.add(layers.GlobalAveragePooling1D())
    model.add(Dense(Num_Class, activation="softmax"))
    model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    return model

## MLP

In [ ]:
def built_model_3(dropout_rate=0.25, activation="relu", Num_Class=None):
    start_neurons = 512
    # create model
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(start_neurons, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 2, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 4, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 8, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate / 2))

    model.add(Dense(Num_Class, activation="softmax"))
    model.compile(
        optimizer="rmsprop",  # 加速神经网络
        loss="categorical_crossentropy",  # 损失函数
        metrics=["accuracy"],
    )
    return model

## USe the follow code to get ADNI_90_120 Data and Label

In [ ]:
import numpy as np
import scipy.io as io

y = io.loadmat("D:/机器学习前沿实验/实验课一/dataset/ADNI_90_120_fMRI.mat")
##AD
AD = np.asarray(y["AD"])
AD_label = np.zeros([AD.shape[0], 1], dtype=int)
##MCI
EMCI = np.asarray(y["EMCI"])
LMCI = np.asarray(y["LMCI"])
MCI = np.vstack((EMCI, LMCI))
MCI_label = np.ones([MCI.shape[0], 1], dtype=int)
##NC
NC = np.asarray(y["NC"])
NC_label = np.full((NC.shape[0], 1), 2, dtype=int)

##合并:
print(AD_label.shape)
print(MCI_label.shape)
print(NC_label.shape)


Data = np.vstack((AD, MCI, NC))
Data = Data[:, :, 10:110]
Label = np.vstack((AD_label, MCI_label, NC_label))
print(Data.shape)
print(Label.shape)

## USe the follow code to get OCD Data and Label

In [ ]:
# import scipy.io as io
# import numpy as np
# import torch
# y=io.loadmat("D:/机器学习前沿实验/实验课一/dataset/OCD_90_200_fMRI.mat")
# FTD=np.asarray(y['OCD'])
# FTD_lable=np.full((FTD.shape[0],1),0,dtype=int)
# NC=np.asarray(y['NC'])
# NC_lable=np.full((NC.shape[0],1),1,dtype=int)
# m=FTD.shape[1]
# n=FTD.shape[2]

# x = FTD.reshape(FTD.shape[0],-1)
# y = NC.reshape(NC.shape[0],-1)

# Label_1=np.vstack((FTD_lable,NC_lable))
# Data_1=np.vstack((x,y))

# Label=Label_1.reshape(Label_1.shape[0])
# Data=(Data_1.reshape(-1,m, n))

## USe the follow code to get FTD Data and Label

In [ ]:
# import scipy.io as io
# import numpy as np
# import torch
# y=io.loadmat("D:/机器学习前沿实验/实验课一/dataset/FTD_90_200_fMRI.mat")
# FTD=np.asarray(y['FTD'])
# FTD_lable=np.full((FTD.shape[0],1),0,dtype=int)
# NC=np.asarray(y['NC'])
# NC_lable=np.full((NC.shape[0],1),1,dtype=int)
# m=FTD.shape[1]
# n=FTD.shape[2]

# x = FTD.reshape(FTD.shape[0],-1)
# y = NC.reshape(NC.shape[0],-1)

# Label_1=np.vstack((FTD_lable,NC_lable))
# Data_1=np.vstack((x,y))

# Data=(Data_1.reshape(-1,m, n))
# Label=Label_1.reshape(Label_1.shape[0])

## Transfer the Label for later training

In [ ]:
from tensorflow.keras.utils import to_categorical

Label = to_categorical(Label)

x = Data.transpose((0, 2, 1))
x = np.asarray(x)

In [ ]:
x.shape, Label.shape

## 五折交叉验证，利用注释处不同代码获得不同效果/不同数据集/是否使用scaledata策略，scaledata策略参数等等

In [ ]:
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=100)  # 5折交叉验证

i = 1
a = []
x = np.asarray(x)

h = ScaleData(x, "powert", "both", 100)  ##进行Scaledata操作使用改代码
# h=x   ##不进行Scaledata操作使用该代码

for train_index, test_index in kf.split(h, Label):
    print("\n{} of kfold {}".format(i, kf.n_splits))
    X_train, X_test = h[train_index], h[test_index]
    y_train, y_test = Label[train_index], Label[test_index]

    ## 使用下面的代码获取模型：built_model_1为LSTM，built_model_2位FCN（一维卷积）built_model_3(为MLP)
    model = built_model_1(Num_Class=3)  ##构建模型时输入类别，ADNI为3，OCD,FTD为2
    history = model.fit(
        X_train,
        y_train,
        batch_size=16,
        epochs=500,
        validation_data=(X_test, y_test),
        callbacks=[reduce_lr],
    )  # 无scale
    x = (np.asarray(history.history["accuracy"]),)
    y = np.asarray(history.history["val_accuracy"])
    b = np.asarray(y)
    print("best accuracy os this circle:", b.max())
    a.append(b.max())
    i += 1
#     model.save(f"1dConv-Fold{i}")  ##是否保存模型

## 打印输出五折交叉验证结果

In [ ]:
a = np.asarray(a)  ##打印输出五折交叉验证结果
print("五折交叉验证结果:", a.mean())